<a href="https://colab.research.google.com/github/ratral/hyd4gpv_py/blob/main/gpv_v002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 0.1 Libreries/Packages
import sys
import numpy as np
import pandas as pd
import plotly.express as px

from pathlib import Path
from scipy.optimize import fsolve
from numpy.core.fromnumeric import mean

# Import General Propuse valve functions functions
sys.path.insert(0,"/content/drive/MyDrive/Colab Notebooks/py")
import hy4gpv as gpv

# Defining the paths for the parameters of the curve
path_valves_data = Path("/content/drive/MyDrive/Colab Notebooks/gpv_data.csv") 

In [2]:
#@title 1.1 Project Operation data Input
  
# Project Operation data
brand      = "VAG" # Brand of the Valve ['VAG' 'ERHARD' 'AVK-ACMO']
diameter   = 0.200    # Valve Diameter in meter
dn_up      = 0.200    # Upstream pipe diameter in meter
dn_down    = 0.200    # Downstream Diameter in meter
plate_up   = 0        # Zeta value of the orifice plate (not yet implemented)
z_plate    = 0        # Zeta value of the orifice plate (not yet implemented)
masl       = 1780     # Elevation of a location in reference to mean sea level.
temp_c     = 15       # Water Temperature in Celsius
safety_factor = 1.3   # The factor of safety for the Kv, this must be >= 1 

# Operation Data:
#   {'p_up': 'bar', 'p_down': 'bar', 'flow': 'm3/h'}
df2 = np.array(
    [["VRF-15_max",  4.439,	2.615,	396.0],
     ["VRF-16_max",	18.263,	5.241,	396.0],
     ["VRF-18_max",	 9.455,	2.615,	396.0],
     ["VRF-23_max",	20.215,	2.190,	396.0],
     ["VRF-24_max",	19.903,	2.190,	396.0],
     ["VRF-25_max",	19.132,	2.190,	396.0],
     ["VRF-26_max",	18.534,	2.347,	396.0],
     ["VRF-27_max",	17.439,	2.347,	396.0],
     ["VRF-28_max",	17.148,	2.347,	396.0],
     ["VRF-34_max",	18.314,	2.718,	396.0],
     ["VRF-15_min",	 4.368,	2.249,	252.0],
     ["VRF-16_min",	18.193,	5.077,	252.0],
     ["VRF-18_min",	 9.337,	2.249,	252.0],
     ["VRF-23_min",	20.185,	2.081,	252.0],
     ["VRF-24_min",	19.873,	2.081,	252.0],
     ["VRF-25_min",	19.100,	2.081,	252.0],
     ["VRF-26_min",	18.470,	2.148,	252.0],
     ["VRF-27_min",	17.372,	2.148,	252.0],
     ["VRF-28_min",	17.081,	2.148,	252.0],
     ["VRF-34_min",	18.299,	2.291,	252.0]]
  )


In [3]:
#@title 1.2 Converting the operation data in a Dataframe and Data Processing

# 1. Converting the operation data in a Dataframe 
# 2. Calculation of the valocity, sigmas, kv and Zeta values
df2 = (
    gpv.convert_to_dataf(df2)
    .pipe(gpv.calculation_operating_data, masl, diameter, temp_c)
)

display(df2.round(3))

,condition,p_up,p_down,flow,p1,p2,dp,velocity,v_factor,sigma_0,sigma_1,sigma_2,kv,zeta
0,VRF-15_max,4.439,2.615,396.0,5.256,3.432,1.824,3.501,0.063,2.872,1.872,1.810,293.974,29.561
1,VRF-16_max,18.263,5.241,396.0,19.080,6.058,13.022,3.501,0.063,1.464,0.464,0.462,110.023,211.044
2,VRF-18_max,9.455,2.615,396.0,10.272,3.432,6.840,3.501,0.063,1.499,0.499,0.495,151.807,110.854
3,VRF-23_max,20.215,2.190,396.0,21.032,3.007,18.025,3.501,0.063,1.166,0.166,0.165,93.515,292.126
4,VRF-24_max,19.903,2.190,396.0,20.720,3.007,17.713,3.501,0.063,1.169,0.169,0.168,94.335,287.070
5,VRF-25_max,19.132,2.190,396.0,19.949,3.007,16.942,3.501,0.063,1.176,0.176,0.176,96.458,274.574
6,VRF-26_max,18.534,2.347,396.0,19.351,3.164,16.187,3.501,0.063,1.194,0.194,0.194,98.682,262.338
7,VRF-27_max,17.439,2.347,396.0,18.256,3.164,15.092,3.501,0.063,1.209,0.209,0.208,102.199,244.592
8,VRF-28_max,17.148,2.347,396.0,17.965,3.164,14.801,3.501,0.063,1.213,0.213,0.212,103.199,239.876
9,VRF-34_max,18.314,2.718,396.0,19.131,3.535,15.596,3.501,0.063,1.226,0.226,0.225,100.534,252.760


In [4]:
#@title 1.3 Reading and preselection of the valves

# Selection of valves that meet the minimum flow coefficient required.
sf = df2.kv.max() * safety_factor

# 1. Read the parameter of the VAG Valves.
# 2. filter brand,  valves diameter and select specific columns
# 3. Calculation of the columns r_coeff,	zvs2,	kvs, and flps
# 4. query the minimum flow coefficient required
valves =  (
    gpv.load_valves_parameter(path_valves_data)
    .pipe(gpv.select_possible_values, brand, diameter)
    .pipe(gpv.calc_zvs_kvs_flps, diameter, dn_up, dn_down, z_plate)
    .query('kvs >= @sf')
)

display(valves.round(3))


,cyl_name,kv_b,kv_d,kv_e,zvs,fls,r_coeff,zvs2,kvs,flps
0,E,-2.926,1.527,80.354,1.900,0.617,0.0,1.900,1159.557,0.617
7,SZ-45,-1.886,5.481,221.491,17.009,0.763,0.0,17.009,387.547,0.763
26,L-45 + E,-3.189,1.203,60.647,17.009,0.769,0.0,17.009,387.547,0.769
33,S-45 + E,-1.965,16.216,399.724,17.009,0.769,0.0,17.009,387.547,0.769
48,SZ-10-30%,-4.271,1.580,88.029,5.560,0.727,0.0,5.560,677.847,0.727
49,SZ-10-50%,-5.433,1.362,82.959,8.966,0.837,0.0,8.966,533.789,0.837
50,SZ-30-20%,-3.907,1.140,60.456,3.144,0.711,0.0,3.144,901.421,0.711
51,SZ-30-50%,-4.670,1.235,73.353,8.104,0.801,0.0,8.104,561.460,0.801
53,LH-15-50%,-3.102,79.694,408.503,9.700,0.723,0.0,9.700,513.196,0.723
54,LH-20-50%,-3.102,79.694,408.503,5.400,0.723,0.0,5.400,687.815,0.723


In [5]:
#@title 1.4 Create combination of the two dataframes and Columns calculation 

# 1. Create combination of the two dataframes valves and df2
# 2. Calculation of the columns kv/kvs	position	sigmas and	regime
# 3. Splitng the Data Frame

valves, df2 = (
    gpv.combination_two_dataframes(valves, df2)
    .pipe(gpv.cal_kvkvs_sigma_regime)
    .pipe(gpv.splitng_operation_data)
)

display(df2.round(3))

print(f'\n')

display(valves.round(3))

,condition,p1,p2,dp,flow,velocity,kv,kv_kvs,zeta,sigma_2,position,sigma_i,sigma_c,sigma_m,regime
cyl_name,,,,,,,,,,,,,,,
E,VRF-15_max,5.256,3.432,1.824,396.0,3.501,293.974,0.254,29.561,1.810,46.283,0.683,0.568,0.412,0
E,VRF-28_min,17.898,2.965,14.933,252.0,2.228,65.381,0.056,597.629,0.197,26.361,0.152,0.126,0.092,0
E,VRF-27_min,18.189,2.965,15.224,252.0,2.228,64.753,0.056,609.275,0.193,26.271,0.151,0.125,0.091,0
E,VRF-26_min,19.287,2.965,16.322,252.0,2.228,62.537,0.054,653.218,0.180,25.949,0.145,0.121,0.088,0
E,VRF-25_min,19.917,2.898,17.019,252.0,2.228,61.243,0.053,681.112,0.169,25.757,0.142,0.118,0.086,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SZ-45,VRF-26_min,19.287,2.965,16.322,252.0,2.228,62.537,0.161,653.218,0.180,34.710,0.229,0.181,0.116,2
SZ-45,VRF-27_min,18.189,2.965,15.224,252.0,2.228,64.753,0.167,609.275,0.193,35.377,0.238,0.188,0.120,1
SZ-45,VRF-34_min,19.116,3.108,16.008,252.0,2.228,63.148,0.163,640.651,0.193,34.895,0.232,0.183,0.117,1


,condition,kv_b,kv_d,kv_e,zvs,fls,r_coeff,kvs,flps
cyl_name,,,,,,,,,
E,VRF-15_max,-2.926,1.527,80.354,1.900,0.617,0.0,1159.557,0.617
E,VRF-28_min,-2.926,1.527,80.354,1.900,0.617,0.0,1159.557,0.617
E,VRF-27_min,-2.926,1.527,80.354,1.900,0.617,0.0,1159.557,0.617
E,VRF-26_min,-2.926,1.527,80.354,1.900,0.617,0.0,1159.557,0.617
E,VRF-25_min,-2.926,1.527,80.354,1.900,0.617,0.0,1159.557,0.617
...,...,...,...,...,...,...,...,...,...
SZ-45,VRF-26_min,-1.886,5.481,221.491,17.009,0.763,0.0,387.547,0.763
SZ-45,VRF-27_min,-1.886,5.481,221.491,17.009,0.763,0.0,387.547,0.763
SZ-45,VRF-34_min,-1.886,5.481,221.491,17.009,0.763,0.0,387.547,0.763


In [7]:
#@title 1.5 cylinder prioritization (summarizing the results)

ser = (
  df2
  .groupby("cyl_name")[["regime","position", "kv_kvs"]]
  .agg([min,max, mean])
)

# Flatten MultiIndex Columns into a Single Index
# https://www.pauldesalvo.com/how-to-flatten-multiindex-columns-into-a-single-index-dataframe-in-pandas/
ser.columns = ['_'.join(col) for col in ser.columns.values]

ser = (
  ser
  .assign(
      kv_kvs_min = ser.kv_kvs_min*100,
      kv_kvs_max = ser.kv_kvs_max*100,
      position_range = ser.position_max - ser.position_min,
      kv_kvs_range = (ser.kv_kvs_max - ser.kv_kvs_min)*100
  )
)


ser = (
  ser
  .assign(
      valve_range =  np.absolute(ser.kv_kvs_range / ser.position_range-1)
  )
)

ser = ser[["regime_min", "regime_max", "regime_mean",
           "position_min", "position_max", "kv_kvs_min", "kv_kvs_max", 
           "position_range", "kv_kvs_range", "valve_range"]]

ser = ser.sort_values(by=["regime_mean", "valve_range"],
                      ascending = [True, True])

display(ser)


,regime_min,regime_max,regime_mean,position_min,position_max,kv_kvs_min,kv_kvs_max,position_range,kv_kvs_range,valve_range
cyl_name,,,,,,,,,,
LH-30-50%,0,0,0.00,40.445473,67.800878,6.104532,30.221907,27.355405,24.117375,0.118369
SZ-10-50%,0,1,0.15,53.141505,77.250239,11.124227,55.073078,24.108734,43.948851,0.822943
SZ-30-20%,0,1,0.20,29.589595,47.838931,6.587365,32.612287,18.249336,26.024921,0.426075
LH-25-50%,0,1,0.25,42.554400,71.348118,7.146137,35.378616,28.793718,28.232479,0.019492
SZ-30-50%,0,1,0.30,44.176324,68.692618,10.575970,52.358802,24.516294,41.782832,0.704288
E,0,2,0.50,25.477249,46.283453,5.120912,25.352266,20.806204,20.231354,0.027629
LH-20-50%,0,2,0.55,45.230973,75.853879,8.633112,42.740231,30.622906,34.107119,0.113778
SZ-10-30%,0,2,0.55,45.322574,70.111699,8.760076,43.368797,24.789125,34.608721,0.396125
LH-15-50%,0,3,1.15,49.715240,83.413593,11.570613,57.283013,33.698353,45.712400,0.356517
